## VAE (Poor Performance)

In [ ]:
class Sampling(layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.seed_generator = random.SeedGenerator(1337)

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = ops.shape(z_mean)[0]
        dim = ops.shape(z_mean)[1]
        epsilon = random.normal(shape=(batch, dim), seed=self.seed_generator)
        return z_mean + ops.exp(0.5 * z_log_var) * epsilon


In [ ]:
inputs = layers.Input(shape=(WIN_SIZE, 2))
x = layers.LSTM(64, return_sequences=True)(inputs)
x = layers.LSTM(32)(x)
z_mean = layers.Dense(16)(x)
z_log_var = layers.Dense(16)(x)
z = Sampling()([z_mean, z_log_var])
encoder = models.Model(inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

In [ ]:
# Decoder here
latent_inputs = layers.Input(shape=(16,))
x = layers.Dense(32)(latent_inputs)
x = layers.RepeatVector(WIN_SIZE)(x)
x = layers.LSTM(64, return_sequences=True)(x)
x = layers.TimeDistributed(layers.Dense(1))(x)
x = layers.Reshape((WIN_SIZE,))(x)
decoder = models.Model(latent_inputs, x, name="decoder")
decoder.summary()

In [ ]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            all_data, price_target = data
            z_mean, z_log_var, z = self.encoder(all_data)
            reconstruction = self.decoder(z)
            reconstruction_loss = ops.mean(
                ops.sum(ops.square(price_target - reconstruction))
            )
            kl_loss = -0.5 * (1 + z_log_var - ops.square(z_mean) - ops.exp(z_log_var))
            kl_loss = ops.mean(ops.sum(kl_loss))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }


In [ ]:
vae = VAE(encoder, decoder)
vae.compile(optimizer=keras.optimizers.Adam())

In [ ]:
# Training the VAE
vae.fit(
    train_windows,
    train_labels,
    epochs=50,
    batch_size=32,
    # validation_split=0.2,
    shuffle=False,
)

vae.evaluate(test_windows, test_labels)